# You do not need to do any of this
This file walks you through the preprocessing steps for:
1. Cleaning the data,
2. Aligning transcripts to utterances at the phoneme level, and
3. Packaging the data for data science and machine learning uses.

**This is a slow, painful, and iterative process. If you're just interested in data science / machine learning, I recommend skipping this and downloading the end results.**

If you want to help out with programmatic data cleaning, then this is for you. There's a lot of work to be done here.

# Convert clipper-formatted data to mfa-formatted data
The goal here is to run Montreal Forced Aligner (MFA) through Clipper's clips. Clipper's files are flac files and word-level transcripts. MFA takes in 16khz wave files and word-level transcripts, and it outputs phoneme-level transcripts. The `datapipes` module in `src/` can convert Clipper's files into MFA-compatible input.

First step: do a dry-run to check for any errors in the Clipper files we have. Sometimes there's a filename mismatch, a missing character name, missing transcript file, or similar. While running this, you'll see the `In [ ]` on the left-hand side change to `In [*]`. When it's complete, you'll see it change to `In [1]`. The number `[1]` tells you the order in which commands on this page were executed.

In [ ]:
!(cd ../src; python -m datapipes --mfa-inputs \
    --input /home/celestia/data/clipper-samples `# clipper-formatted directory` \
    --output /home/celestia/data/mfa-inputs `# mfa-formatted directory` \
    --delta `# ignore files already processed` \
    --dry-run `# don't create any output files`) | nl

If there are any errors, make sure to fix them and re-run the above command. Repeat until there are no errors, then run the next command to generate the mfa-formatted data. If you're running this on all of Clipper's data, this might take an hour to complete.

In [ ]:
!(cd ../src; python -m datapipes --mfa-inputs \
    --input "/home/celestia/data/clipper-samples" \
    --output /home/celestia/data/mfa-inputs \
    --delta)

Finally, run montreal-forced-aligner with the following command to generate phoneme-level transcripts. Note that, due to quirks with IPython, this command won't produce intermediate output, so you won't be able to monitor progress here. If you're running this on all of Clipper's data, this command might take a few hours to complete. You can monitor progress by watching the `data/mfa-alignments` directory.

In [ ]:
%%bash

rm -r /home/celestia/data/mfa-alignments
rm -r /home/celestia/Documents/MFA

function mfa() {
    mkdir /home/celestia/data/mfa-alignments/$1 || true
    yes n | mfa_align -v `# continue even with an incomplete dictionary` \
        /home/celestia/data/mfa-inputs/$1 `# input directory` \
        /opt/mfa/pronunciations_dicts/english.dict.txt \
        /opt/mfa/pretrained_models/english.zip \
        /home/celestia/data/mfa-alignments/$1 `# output directory` \
        || true
}

export -f mfa

ls /home/celestia/data/mfa-inputs | xargs -L1 -P16 bash -c 'mfa $@' _

It's extremely likely that MFA failed on some inputs. There are three ways in which it can fail:
1. MFA found a word it didn't recognize and logged both the missing word and corresponding utterance.
2. MFA failed in some unexpected way while doing preprocessing for a character, and it borked its own configuration files.
3. MFA couldn't figure out how to align the transcript to an utterance.

For the first kind of failure, you can find an `oovs_found.txt` file in each of the directories within `mfa-alignments`. This file contains a list of words that could not be processed because they don't exist in the pronunciation dictionary. You can find the current pronunciation dictionary in `/opt/mfa/pronunciations_dicts/english.dict.txt`. If you end up adding the pronunciations of any missing words, make sure to post them to the thread. I can update the Docker image so everyone can benefit from it.

In [ ]:
! cat /home/celestia/data/mfa-alignments/*/oovs_found.txt

For the second and third kinds of failure, you can find out which characters MFA failed to process by searching for the empty directories  in `mfa-alignments`.

In [ ]:
! find /home/celestia/data/mfa-alignments -type d -empty

If the above command produces any output, it's very likely that MFA stochastically borked something during its own preprocessing stage. The easiest way to handle this is to remove its character-specific cache directory and try again.

The following script does exactly that for the case where MFA fails on Applejack's files. In my case, I needed to run this for Apple-Bloom, Applejack, Cadance, and Rainbow-Dash the most recent time, but MFA's failures are pretty stochastic.

In [ ]:
%%bash

retry_character="AK-Yearling"

rm -r "/home/celestia/Documents/MFA/$retry_character"

yes n | mfa_align -v \
        /home/celestia/data/mfa-inputs/$retry_character \
        /opt/mfa/pronunciations_dicts/english.dict.txt \
        /opt/mfa/pretrained_models/english.zip \
        /home/celestia/data/mfa-alignments/$retry_character

The last type of MFA failure is the only one that's complicated to handle. If you run the following command, you can see a list of transcripts that MFA failed to align.

In [ ]:
%%bash

function get_textgrids() {
    (cd "$1"
    find -iname '*.textgrid' |
        sed 's/\.textgrid$//gI' |
        sort)
}

diff <(get_textgrids /home/celestia/data/mfa-inputs) <(get_textgrids /home/celestia/data/mfa-alignments)

If you didn't complete the above steps for handling whole-character issues, you'll notice that some characters have a huge number of utterances listed. If you did complete the above steps, none of the characters should have _that_ many failures. For me, the worst offender is Pinkie Pie with 77 failures, followed by Fluttershy and Twilight Sparkle both with around 35. If a character has a huge number of utterances listed, it's likely that MFA crashed at some point. You can read through its logs in `/home/celestia/Documents/MFA/` to try to figure out why. 

You can try playing some of the listed files to figure out why MFA might be failing on them. I've found that it's often because either (1) the character is speaking in a very excited or abnormal way, (2) the clip is noisy or muffled, or (3) the utterance contains a lot of out-of-dictionary words.

Eventually, we'll want to find a way to make use of these utterances to generate more realistic speech in niche cases, but for now we can ignore them.

# Packaging data


In [ ]:
!(cd ../src; python -m datapipes --audio-tar \
    --input-audio /home/celestia/data/clipper-samples `# clipper-formatted directory` \
    --input-alignments /home/celestia/data/mfa-alignments `# mfa-formatted directory` \
    --output /home/celestia/data/audio-tar `# output per-character tar.gz files here` \
    --audio-format 'wav' \
    --sampling-rate 48000 \
    --dry-run `# don't create any output files` \
    --verbose)

In [ ]:
!(cd ../src; python -m datapipes --audio-tar \
    --input-audio /home/celestia/data/clipper-samples \
    --input-alignments /home/celestia/data/mfa-alignments \
    --output /home/celestia/data/audio-tar \
    --audio-format 'wav' \
    --sampling-rate 48000 \
    --verbose)

In [ ]:
# test the archives

# Additional processing

In [ ]:
%%bash

function generate_info() {
    source="$(readlink -f $1)"
    target="/home/celestia/data/audio-info/$(basename $source .tar).txz"
    (cd ../src; python -m datapipes --audio-info \
        --input-tar "$source" \
        --output-txz "$target" \
        --verbose) || true
}

export -f generate_info

ls /home/celestia/data/audio-tar/* | xargs -L1 -P16 bash -c 'generate_info $@' _

In [23]:
%%bash

function generate_tfrecords() {
    audio="$(readlink -f $1)"
    info="/home/celestia/data/audio-info/$(basename $audio .tar).txz"
    tfrecord="/home/celestia/data/audio-tfrecord/$(basename $audio .tar).tfrecord"
    (cd ../src; python -m datapipes --audio-record \
        --input-audio "$audio" \
        --input-info "$info" \
        --output-tfrecord "$tfrecord" \
        --verbose) || true
}

export -f generate_tfrecords

ls /home/celestia/data/audio-tar/* | xargs -L1 -P16 bash -c 'generate_tfrecords $@' _

Generating /home/celestia/data/audio-tfrecord/Big-Bucks.tfrecord
Done
Generating /home/celestia/data/audio-tfrecord/Apple-Cobbler.tfrecord
Done
Generating /home/celestia/data/audio-tfrecord/All-Aboard.tfrecord
Done
Generating /home/celestia/data/audio-tfrecord/Biscuit.tfrecord
Done
Generating /home/celestia/data/audio-tfrecord/Aria-Blaze.tfrecord
Done
Generating /home/celestia/data/audio-tfrecord/Ahuizotl.tfrecord
Done
Generating /home/celestia/data/audio-tfrecord/Apple-Rose.tfrecord
Done
Generating /home/celestia/data/audio-tfrecord/Auntie-Applesauce.tfrecord
Done
Generating /home/celestia/data/audio-tfrecord/Autumn-Blaze.tfrecord
Done
Generating /home/celestia/data/audio-tfrecord/Babs-Seed.tfrecord
Done
Generating /home/celestia/data/audio-tfrecord/Blaze.tfrecord
Done
Generating /home/celestia/data/audio-tfrecord/Big-Daddy-Mccolt.tfrecord
Done
Generating /home/celestia/data/audio-tfrecord/Boyle.tfrecord
Done
Generating /home/celestia/data/audio-tfrecord/Bright-Mac.tfrecord
Done
Gener

2019-12-29 08:11:36.697761: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-12-29 08:11:36.727311: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX TITAN Z major: 3 minor: 5 memoryClockRate(GHz): 0.8755
pciBusID: 0000:03:00.0
2019-12-29 08:11:36.728743: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: GeForce GTX TITAN Z major: 3 minor: 5 memoryClockRate(GHz): 0.8755
pciBusID: 0000:04:00.0
2019-12-29 08:11:36.729855: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 2 with properties: 
name: GeForce GTX TITAN Z major: 3 minor: 5 memoryClockRate(GHz): 0.8755
pciBusID: 0000:07:00.0
2019-12-29 08:11:36.730174: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2019-12-29 08:11:36.731716: I tensorflow/stream_executor/platform/defau

In [ ]:
%%bash

function trim_character() {
    input_tar="$(readlink -f $1)"
    output_folder="/home/celestia/data/audio-trimmed/$(basename $input_tar .tar)"
    echo "writing to $output_folder"
    mkdir -p "$output_folder"
    (cd "$output_folder"
        tar xf "$input_tar";
        for clip in `ls`; do
            cp "$clip/audio.wav" "tmp.wav";
            sox "tmp.wav" "$clip/audio.wav" silence 1 0.05 0.1% reverse silence 1 0.05 0.1% reverse;
        done
        rm "tmp.wav"
    )
}

export -f trim_character

ls /home/celestia/data/audio-tar-22khz/* | xargs -L1 -P16 bash -c 'trim_character $@' _

In [ ]:
ls | while read line; do (cd "$line"; tar -cf ../$line.tar */audio.wav */label.json);